In [ ]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
import graphviz
from sklearn.tree import export_graphviz
from IPython.display import SVG
import numpy as np
import json
import regex as re
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
data = pd.read_csv('gdrive/MyDrive/Hahaton/train.csv')
# data

In [ ]:
data['label'].unique()

array([ 0.,  1., nan])

In [ ]:
dataNotNan = data.loc[data['label'] != np.NaN]

In [ ]:
str = data['text'][2]

In [ ]:
parceOfEnter = str.split("\n")
# parceOfEnter

In [ ]:
letal = data.loc[data['label'] == 1]
# letal['text'][254]



In [ ]:
# TODO: нужен отдельный конфиг
diagnoses = ['ДИАГНОЗ_ПОСТУПЛЕНИЯ_ОСНОВНОЙ', 
             'ДИАГНОЗ_ПОСТУПЛЕНИЯ_СОПУТСТВУЮЩИЙ', 
             'ДИАГНОЗ_ПОСТУПЛЕНИЯ_ОСЛОЖНЕНИЕ',
             'Пищеварительная система',
             'Хронические заболевания в анамнезе',
             'Сердечно-сосудистая система',
             'Органы дыхания',
             'Наследственность']

mapObjects = [] 
out = []
setKeys = {}

p = re.compile(r'[;]|\s[А-ЯЁ]|\s[,]\s')
pUpper = re.compile(r'[А-ЯЁ]')

for index, row in data.iterrows():
  mapObject = {'id':row['id'], 'label':row['label']}
  # print(type(row['text']))
  if (type(row['text']) == type(str)):
    splitedRow = row['text'].split('\n')
    for line in splitedRow:
      if (line.find(':') > 0):
        splitedLine = line.split(':', 1)
        # print(splitedLine)
        if (splitedLine[0].strip() in diagnoses):
          diag = p.split(splitedLine[1])
          out = diag
          for l in diag:
            if (line.find(l) > 0 and pUpper.match(line[line.find(l) - 1])):
              l = line[line.find(l) - 1] + l
            if (l.find(':') > 0):
              # print(l)
              splitedL = l.split(':', 1)
              mapObject[splitedL[0].strip()] = splitedL[1].strip()
            else:
              mapObject[l] = True
        else:
          mapObject[splitedLine[0].strip()] = splitedLine[1].strip()
      else:
        mapObject[line] = True
    
  mapResult = {}
  for key in mapObject:
    if (mapObject[key] != '' and key != '' and len(key) < 40):
      mapResult[key.strip().strip('.')] = mapObject[key].strip(',')
      key = key.strip()
      if key not in setKeys:
        setKeys[key] = 1
      else:
        setKeys[key] = setKeys[key] + 1

  mapObjects.append(mapResult)

In [ ]:
newSetKeys = {}
for key in setKeys:
    if setKeys[key] > 30:
      newSetKeys[key] = setKeys[key]

mapResults = []
for obj in mapObjects:
  newObj = {}
  for key in obj:
    if key in newSetKeys:
      newObj[key] = obj[key]
  mapResults.append(newObj)

In [ ]:
# len(mapResults)
# mapResults[0]

79336

In [ ]:
numPat = re.compile(r'^[1-9][0-9]*(?:\.[0-9]+)?')
flagSet = {}

for key in newSetKeys:
  flagSet[key] = True

for obj in mapResults:
  # print(obj)
  for key in obj:
    # print(key)
    if (flagSet[key] and type(obj[key]) == type(str) and not numPat.match(obj[key])):
      flagSet[key] = False

keysWithNums = []
for key in flagSet:
  if flagSet[key]:
    keysWithNums.append(key)

id
label
Инъекции 01.2016 ФМИЦ
ИБС СН 2фк
I20.8
Другие формы стенокардии
ИБС.
Стенокардия напряжения II
Гипертоническая болезнь III ст.
Риск 4.
Пролапс
МК.
МН 2 ст.
Транзиторная
АВ-блокада 1 ст.
Гиперлипидемия.
НТГ.
МКБ.
Хр.пиелонефрит.
гастрит хронический
Сознание ясное
В пространстве и времени ориентирован
Склеры обычной_окраски
Число дыханий в 1 мин. 15
Грудная клетка правильной_формы
Дистанционные хрипы не_слышны
дыхание жесткое
Язык влажный
Печень не_увеличена
край печени умеренной_плотности
поверхность печени гладкая
Бессимптомная ишемия миокарда
Безболевая ишемия.
Атеросклероз
Хр.гастрит, ремиссия.
Эутиреоз.
Хр.пиелонефрит, ремиссия.
I70.8
Атеросклероз других артерий
I50.9
Сердечная недостаточность неуточненная
язвенная болезнь желудка ремиссия
В контакт вступает_легко
Эмоциональная лабильность не_выражена
Число дыханий в 1 мин. 16
дыхание везикулярное
Стенокардия напряжения II ф.кл.
Риск
ССО 4.
Желудочковая экстрасистолия.
Хронический пиелонефрит, ремиссия.
M42.1
Остеохондроз п

In [ ]:
dataForTree = pd.DataFrame(mapResults)
X = dataForTree.drop(columns=['label']) # все кроме label
# y = dataForTree['label'].apply(str)

# clf = DecisionTreeClassifier(criterion='entropy', min_samples_leaf=5)
# clf.fit(X, dataForTree['label'])